In [ ]:
!pip install catboost

In [1]:
import pandas as pd
%matplotlib inline
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
# Load training data, reshape, add departure time as an integer number of seconds and add day of week:
df = pd.read_csv('../TotalEnergies/data/Train.csv', parse_dates=['travel_date'], dayfirst=True)
train = df.groupby(['ride_id', 'travel_date', 'travel_time', 'travel_from', 'max_capacity']).size().reset_index(name='Count') #sort=False if needed?
train["travel_time"] = train["travel_time"].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
train['day'] = train['travel_date'].dt.dayofweek

C:\Users\michael\AppData\Local\Temp\ipykernel_11424\1301998871.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('../TotalEnergies/data/Train.csv', parse_dates=['travel_date'], dayfirst=True)


In [3]:
# The same for the test data
test = pd.read_csv('../TotalEnergies/data/Test.csv', parse_dates=['travel_date'], dayfirst=True).drop(['car_type', 'travel_to'], axis=1)
test["travel_time"] = test["travel_time"].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
test['day'] = test['travel_date'].dt.dayofweek

C:\Users\michael\AppData\Local\Temp\ipykernel_11424\924476196.py:2: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  test = pd.read_csv('../TotalEnergies/data/Test.csv', parse_dates=['travel_date'], dayfirst=True).drop(['car_type', 'travel_to'], axis=1)


In [4]:
# The sample submission file
sample = pd.read_csv('../TotalEnergies/submissions/SampleSubmission.csv')

In [5]:
# Combine training and test data for now, so that we can add uber movement data all in one go
train['t'] = 0
test['t'] = 1
X = pd.concat([train, test], sort=False)

In [6]:
# Load travel times from Uber movement data ( 3 x 3month periods)
"""
t1 = pd.read_csv('../TotalEnergies/data/Travel_Times_Daily_1.csv',parse_dates=['Date'])
t2 = pd.read_csv('../TotalEnergies/data/Travel_Times_Daily_2.csv',parse_dates=['Date'])
t3 = pd.read_csv('../TotalEnergies/data/Travel_Times_Daily_3.csv',parse_dates=['Date'])
"""

t4 = pd.read_csv('../TotalEnergies/data/Travel_Times_Awendo.csv',parse_dates=['Date'])
t5 = pd.read_csv('../TotalEnergies/data/Travel_Times_HomaBay.csv',parse_dates=['Date'])
t6 = pd.read_csv('../TotalEnergies/data/Travel_Times_Kehancha.csv',parse_dates=['Date'])
t7 = pd.read_csv('../TotalEnergies/data/Travel_Times_KenduBay.csv',parse_dates=['Date'])
t8 = pd.read_csv('../TotalEnergies/data/Travel_Times_Keroka.csv',parse_dates=['Date'])
t9 = pd.read_csv('../TotalEnergies/data/Travel_Times_Keumbu.csv',parse_dates=['Date'])
t10 = pd.read_csv('../TotalEnergies/data/Travel_Times_Kijauri.csv',parse_dates=['Date'])
t11 = pd.read_csv('../TotalEnergies/data/Travel_Times_Kisii.csv',parse_dates=['Date'])
t12 = pd.read_csv('../TotalEnergies/data/Travel_Times_Mbita.csv',parse_dates=['Date'])
t13 = pd.read_csv('../TotalEnergies/data/Travel_Times_Migori.csv',parse_dates=['Date'])
t14 = pd.read_csv('../TotalEnergies/data/Travel_Times_Ndhiwa.csv',parse_dates=['Date'])
t15 = pd.read_csv('../TotalEnergies/data/Travel_Times_Nyachenge.csv',parse_dates=['Date'])
t16 = pd.read_csv('../TotalEnergies/data/Travel_Times_Oyugis.csv',parse_dates=['Date'])
t17 = pd.read_csv('../TotalEnergies/data/Travel_Times_Rodi.csv',parse_dates=['Date'])
t18 = pd.read_csv('../TotalEnergies/data/Travel_Times_Rongo.csv',parse_dates=['Date'])
t19 = pd.read_csv('../TotalEnergies/data/Travel_Times_Sirare.csv',parse_dates=['Date'])
t20 = pd.read_csv('../TotalEnergies/data/Travel_Times_Sori.csv',parse_dates=['Date'])



travel_times = pd.concat([t4, t5, t6, t7, t8, t9, t10, t11, t12, t13, t14, t15, t16, t17, t18, t19, t20], ignore_index=True)
travel_times = travel_times.fillna(method='ffill')[['Daily Mean Travel Time (Seconds)', 'Date']]
travel_times['Date'] = pd.to_datetime(travel_times['Date'])
travel_times.head(1)

ValueError: Missing column provided to 'parse_dates': 'Date'

In [13]:
# Merge with our contest data
X['Date'] = X['travel_date']
X.set_index('travel_date', inplace=True)
X = X.merge(travel_times, how='left', on='Date')
X.head(1)

,ride_id,travel_time,travel_from,max_capacity,Count,day,t,Date,Daily Mean Travel Time (Seconds)
0,1442,435,Migori,49,1.0,1,0,2017-10-17,2698.0


In [20]:
# Create the model
model = CatBoostRegressor(iterations=200, 
                          depth=4, 
                          learning_rate=0.5, 
                          loss_function='MAE', verbose=False)

In [22]:
in_cols = ['travel_time', 'travel_from', 'max_capacity', 'day'] #'Daily Mean Travel Time (Seconds)' as an option

In [23]:
# Train model
tr = X.loc[X.t == 0]
model.fit(tr[in_cols], tr['Count'], cat_features=['travel_from', 'max_capacity', 'day'])

In [24]:
# Score model
print(mean_absolute_error(model.predict(tr[in_cols]), tr['Count']))

3.499389595249243


In [25]:
# Make predictions and append to the sample submission data, and save as csv
te = X.loc[X.t == 1]
te[in_cols].head()
te = X.loc[X.t == 1]
sample['number_of_ticket'][5:] = model.predict(te[in_cols])[5:] # Ignore the warning
sample.to_csv('catboost_predictions.csv', index=False)
sample.head(10)

,ride_id,number_of_ticket
0,4446,0.000000
1,13962,0.000000
2,5569,0.000000
3,1675,0.000000
4,5711,0.000000
5,2417,9.778770
6,15010,14.936669
7,1823,9.622466
8,15191,9.856025
9,14402,1.651639
